In [5]:
import src.utils as utils
import os
import torch
import shutil

from src.transforms import clipped_zoom
from PIL import Image

### Split train-val and test

Split data into (train & validaiton) and test with 0.8:0.2 ratio.
Clean the data to remove entries with invalid objects

In [ ]:
# Split between train and test for mask and image in raw data
name = "Sophia"
# Move fraction of train data to test folder
source_directory = f"data/{name}/data_train/data_original/masks"
source_directory2 = f"data/{name}/data_train/data_original/images"

destination_directory = f"data/{name}/data_test/masks"
destination_directory2 = f"data/{name}/data_test/images"
fraction_to_move = 0.2

split_train_test_mask(source_directory, destination_directory, source_directory2, destination_directory2, fraction_to_move)

In [ ]:
# move all data to clean data before cleaning
# Data path to image and masks
data_path_raw = f'data/{name}/data_raw/data_original/data_train'
test_path_raw = f'data/{name}/data_raw/data_original/data_test'
data_path = f'data/{name}/data_clean/data_original/data_train'
test_path = f'data/{name}/data_clean/data_original/data_test'

move_images(data_path_raw, data_path, all_ = True)
move_images(test_path_raw, test_path, all_ = True)

In [ ]:
# Clean data
dataset = DEMDataset(data_path_raw, n_channel = 1, channel_list = ["images"])
test_set = DEMDataset(test_path_raw, n_channel = 1, channel_list = ["images"])
original_data = DEMDataset(data_path_raw, n_channel = 1, channel_list = ["images"])
original_test = DEMDataset(test_path_raw, n_channel = 1, channel_list = ["images"])
# Clean data and generate clean file directory 
dataset, test_set = utils.clean_raw_data(dataset, test_set, original_data, original_test, test_path, data_path)

In [204]:
# Add slope, aspect data and split into train/ test
# Move / split slope and aspect data according to mask and image directories
# List all names in cleaned directory
tile_train = [item for item in os.listdir(os.path.join(data_path, "masks"))
                           if item.endswith(".tif")]
check_train = [item for item in os.listdir(os.path.join(data_path, "images"))
                           if item.endswith(".tif")]
tile_test = [item for item in os.listdir(os.path.join(test_path, "masks"))
                           if item.endswith(".tif")]
check_test = [item for item in os.listdir(os.path.join(test_path, "images"))
                           if item.endswith(".tif")]
if (check_train != tile_train) | (check_test != tile_test):
    print("Warning, data has wrongly been added to the clean directory. Mask and image tiles do not correspond. Repeat data splitting and cleaning part")

# Slope train
slope_path_raw = f"data/{name}/data_raw/slope"
slope_path_clean = f"data/{name}/data_clean/data_train/slope"
move_images(slope_path_raw, slope_path_clean, all_ = False, to_move = tile_train)

# slope test
slope_test_clean = f"data/{name}/data_clean/data_test/slope"
move_images(slope_path_raw, slope_test_clean, all_ = False, to_move = tile_test)

# aspect train
aspect_path_raw = f"data/{name}/data_raw/aspect"
aspect_path_clean = f"data/{name}/data_clean/data_train/aspect"
move_images(aspect_path_raw, aspect_path_clean, all_ = False, to_move = tile_train)

# aspect test
aspect_test_clean = f"data/{name}/data_clean/data_test/aspect"
move_images(aspect_path_raw, aspect_test_clean, all_ = False, to_move = tile_test)

***
### Data augmentation

In [260]:
def clean_zoomed_data(dataset, original_data, data_path):
    '''
    Delete dataset in data_clean folder that are invlid by using delete property generated in __getitem__ from file dictionary
    Invalid data: images that are empty (full of 0 or NaN) or corresponding mask are empty
    
    Input:
        dataset, test_set: DEM dataset generated from raw data, will be cleaned in this function through implementation in  __getitem__ 
        original_data, original_test: DEM dataset generated from raw data, uncleaned
        data_path: Path of tobe cleaned directory
    
    '''
    # Delete invalid entries from DEMDataset: __getitem__ deletes invalid entries == a one time iteration 
    for file in dataset:
        pass
    
    # Calculate intersection between original data and cleaned data 
    intersection_set = set(original_data.filename_list).intersection(dataset.filename_list)
    
    # Find the entries that are not in the intersection
    not_in_intersection = [entry for entry in original_data.filename_list + dataset.filename_list if entry not in intersection_set]
    
    # delete file form data_clean folder which are not in the intersection = data deleted in for loop above
    for file in not_in_intersection:
        delete_img = os.path.join(data_path, "images", file)
        if os.path.exists(delete_img):
            # Delete the file
            os.remove(delete_img)
            print(f"{file} image deleted successfully.")
        else:
            print(f"{file} image not found. No deletion performed.")
            
        delete_mask = os.path.join(data_path, "masks", file)
        if os.path.exists(delete_mask):
            os.remove(delete_mask)
            print(f"{file} mask deleted successfully.")
        else:
            print(f"{file} mask not found. No deletion performed.")
            
        delete_aspect = os.path.join(data_path, "aspect", file)
        if os.path.exists(delete_aspect):
            os.remove(delete_aspect)
            print(f"{file} aspect deleted successfully.")
        else:
            print(f"{file} aspect not found. No deletion performed.")
            
        delete_slope = os.path.join(data_path, "slope", file)
        if os.path.exists(delete_slope):
            os.remove(delete_slope)
            print(f"{file} slope deleted successfully.")
        else:
            print(f"{file} slope not found. No deletion performed.")

In [13]:
# Generate transformed data. For each transformation, a new dataset is extracted from train_set (50%). This dataset is augmented. 

random.seed(seed_value)
torch.manual_seed(seed_value)

path_original = f"data/{name}/data_clean/data_train/data_original"
channel_t = ["images", "masks", "aspect", "slope"]
percentage = 0.5

# Horizontal flip transformation-----------------------------------------
tobe_transformed, _ = split_dataset_by_percentage(train_set, percentage)
for channel in channel_t:
    # Transform each image/ channel
    for _, targ in tobe_transformed:
        # Path to original image
        path_tobe_transf = os.path.join(path_original, channel, targ['tile'])

        # Path to file directory save
        path_transformation = f"data/{name}/data_clean/data_train/data_transformed/h_flipping"
        path_save = os.path.join(path_transformation, channel, targ['tile'])

        # Horizontal flip & save image
        Image.open(path_tobe_transf).transpose(Image.FLIP_LEFT_RIGHT).save(path_save, format="TIFF")
        
# Vertical flip transformation-----------------------------------------
tobe_transformed, _ = split_dataset_by_percentage(train_set, percentage)
for channel in channel_t:
    # Transform each image/ channel
    for _, targ in tobe_transformed:
        # Path to original image
        path_tobe_transf = os.path.join(path_original, channel, targ['tile'])

        # Path to file directory save
        path_transformation = f"data/{name}/data_clean/data_train/data_transformed/v_flipping"
        path_save = os.path.join(path_transformation, channel, targ['tile'])

        # Vertical flip & save image
        Image.open(path_tobe_transf).transpose(Image.FLIP_TOP_BOTTOM).save(path_save, format="TIFF")

        
# Zoom in by 1.25
tobe_transformed, _ = split_dataset_by_percentage(train_set, percentage)
for channel in channel_t:
    round = 0
    # Transform each image/ channel
    for _, targ in tobe_transformed:

        # Path to original image
        path_tobe_transf = os.path.join(path_original, channel, targ['tile'])

        # Path to file directory save
        path_transformation = f"data/{name}/data_clean/data_train/data_transformed/zoom"
        path_save = os.path.join(path_transformation, channel, targ['tile'])

        # zoom & save image
        zm = clipped_zoom(np.array(Image.open(path_tobe_transf)), 1.25, channel)
        zoomed_img = Image.fromarray(zm)
        zoomed_img.save(path_save, format="TIFF")

# Zoom out by 0.75
tobe_transformed, _ = split_dataset_by_percentage(train_set, percentage)
for channel in channel_t:
    round = 0
    # Transform each image/ channel
    for _, targ in tobe_transformed:
        '''
        if round == 10:
            break
        round+=1'''
        # Path to original image
        path_tobe_transf = os.path.join(path_original, channel, targ['tile'])

        # Path to file directory save
        path_transformation = f"data/{name}/data_clean/data_train/data_transformed/zoom_out"
        path_save = os.path.join(path_transformation, channel, targ['tile'])

        # zoom & save image
        zm = clipped_zoom(np.array(Image.open(path_tobe_transf),dtype = float), 0.75, channel)
        zoomed_img = Image.fromarray(zm)
        zoomed_img.save(path_save, format="TIFF")


In [15]:
# Change image names so that it can be differentiated to original image 
channel_t = ["images", "masks", "aspect", "slope"]
transformation =  ["h_flipping", "v_flipping", "zoom", "zoom_out"]
path_directory = "data/Sophia/data_clean/data_train/data_transformed"
for transformation_ in transformation:
    for channel in channel_t:
        path_d = os.path.join(path_directory, transformation_, channel)
        file_list = os.listdir(path_d)
        for filename in file_list:
            tilename, img_form = os.path.splitext(filename)
            if transformation_ == "h_flipping":
                addon = '_hflip'
            elif transformation_ == "v_flipping":
                addon = '_vflip'
            elif transformation_ == "zoom":
                addon = '_zoom'
            elif transformation_ == "zoom_out":
                addon = '_zoomout'
            else:
                break
            new_filename = f'{tilename}{addon}{img_form}'
            old_path = os.path.join(path_d, filename)
            new_path = os.path.join(path_d, new_filename)
            os.rename(old_path, new_path)

In [274]:
# Clean unvalid data
# Only zoom in can generate unvalid data (No longer contains RTS) -> Clean zoom data

#data_root = f'data/{name}/data_clean/data_train/data_original_plus_transformed' #data_transformed/zoom'
data_root = 'data/Sophia/data_clean/data_test_tuktoyaktuk'

# Clean data
toclean_zoom = DEMDataset(data_root, n_channel = 1, channel_list = ["images"])
original_zoom = DEMDataset(data_root, n_channel = 1, channel_list = ["images"])


In [289]:
clean_zoomed_data(toclean_zoom, original_zoom, data_root)

2010-2016_tile_17_40_10_6_8_7.tif image deleted successfully.
2010-2016_tile_17_40_10_6_8_7.tif mask deleted successfully.
2010-2016_tile_17_40_10_6_8_7.tif aspect deleted successfully.
2010-2016_tile_17_40_10_6_8_7.tif slope deleted successfully.
2010-2016_tile_17_40_9_7_2_8.tif image deleted successfully.
2010-2016_tile_17_40_9_7_2_8.tif mask deleted successfully.
2010-2016_tile_17_40_9_7_2_8.tif aspect deleted successfully.
2010-2016_tile_17_40_9_7_2_8.tif slope deleted successfully.
2010-2016_tile_18_40_2_6_0_0.tif image deleted successfully.
2010-2016_tile_18_40_2_6_0_0.tif mask deleted successfully.
2010-2016_tile_18_40_2_6_0_0.tif aspect deleted successfully.
2010-2016_tile_18_40_2_6_0_0.tif slope deleted successfully.


In [35]:
# Copy data over from original
root_path = 'data/Sophia/data_clean/data_train/data_original'
target_path = 'data/Sophia/data_clean/data_train/data_original_plus_transformed'
channel_ = ["images", "masks", "aspect", "slope"]
for channel in channel_:
    path_directory = os.path.join(root_path, channel)
    for file_name in  os.listdir(path_directory):
        path_origin = os.path.join(root_path, channel,file_name)
        path_target = os.path.join(target_path, channel,file_name)
        shutil.copy(path_origin, path_target)

In [40]:
# Copy data over from transformed
root_path = 'data/Sophia/data_clean/data_train/data_transformed'
target_path = 'data/Sophia/data_clean/data_train/data_original_plus_transformed'
transformation = ["h_flipping", "v_flipping", "zoom", "zoom_out"]
channel_ = ["images", "masks", "aspect", "slope"]
for transformation_ in transformation:
    for channel in channel_:
        path_directory = os.path.join(root_path, transformation_, channel)
        tot_filename = [item for item in os.listdir(path_directory)
                           if item.endswith(".tif")]
        for file_name in tot_filename:
            path_origin = os.path.join(root_path, transformation_, channel,file_name)
            path_target = os.path.join(target_path, channel,file_name)
            shutil.copy(path_origin, path_target)

#### Clean data


    

In [7]:
# Remove bad directories
'''
channel_t = ["images", "masks", "aspect", "slope"]
transformation = ["h_flipping", "v_flipping", "zoom", "zoom_out"]
path_delete = "data/Sophia/data_clean/data_train/data_transformed"
for transformation_ in transformation:
    for channel in channel_t:
        path_d = os.path.join(path_delete, transformation_, channel)
        file_list = os.listdir(path_d)

        # Iterate over the files and delete each one
        for file_name in file_list:
            file_path = os.path.join(path_d, file_name)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    #print(file_path)
                    
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")
'''

'\nchannel_t = ["images", "masks", "aspect", "slope"]\ntransformation = ["h_flipping", "v_flipping", "zoom", "zoom_out"]\npath_delete = "data/Sophia/data_clean/data_train/data_transformed"\nfor transformation_ in transformation:\n    for channel in channel_t:\n        path_d = os.path.join(path_delete, transformation_, channel)\n        file_list = os.listdir(path_d)\n\n        # Iterate over the files and delete each one\n        for file_name in file_list:\n            file_path = os.path.join(path_d, file_name)\n            try:\n                if os.path.isfile(file_path):\n                    os.remove(file_path)\n                    #print(file_path)\n                    \n            except Exception as e:\n                print(f"Error deleting {file_path}: {e}")\n'

In [ ]:
# move data
import shutil
origin = 'data/Sophia/data_clean/data_train/data_original'
target = 'data/Sophia/data_clean/data_train/data_original_plus_transformed'
shutil.copy(origin, target)

## Circular encoding of aspect data

In [769]:
root_pt_list = [f'data/{name}/data_clean/data_train/data_original', f'data/{name}/data_clean/data_train/data_original_plus_transformed', 
                f'data/{name}/data_clean/data_train/data_transformed/h_flipping', f'data/{name}/data_clean/data_train/data_transformed/v_flipping', 
               f'data/{name}/data_clean/data_train/data_transformed/zoom', f'data/{name}/data_clean/data_train/data_transformed/zoom_out',
               f'data/{name}/data_clean/data_test']

In [772]:
for root_pt in root_pt_list:
    tile_name = [item for item in os.listdir(os.path.join(root_pt, "aspect"))
                               if item.endswith(".tif")]
    round = 0
    for tile in tile_name:
        '''
        if round== 5:
            break
        round+=1
        '''
        # path to aspect image
        path_tobe_transf = os.path.join(root_pt, "aspect",tile)
        # Path to file directory save
        path_x_aspect = os.path.join(root_pt, 'x_aspect', tile)
        path_y_aspect = os.path.join(root_pt, 'y_aspect', tile)

        # Get data
        matrix_aspect = np.array(Image.open(path_tobe_transf))
        # Smooth image to get overall large pattern
        smoothed_matrix = gaussian_filter(matrix_aspect, sigma=15)
        # Circular encoding
        x_aspect = np.cos(smoothed_matrix)
        y_aspect = np.sin(smoothed_matrix)

        # save image
        Image.fromarray(x_aspect).save(path_x_aspect, format="TIFF")
        Image.fromarray(y_aspect).save(path_y_aspect, format="TIFF")

***
### Create directory only filled with 2021 data if it is also in 2016

In [ ]:
import shutil
channel_list = ['images', 'masks', 'slope']
dir1 = "data/Sophia/data_clean/data_test"
dir2 = "data/Sophia/data_clean/2010_2021"
dir3 = "data/Sophia/data_clean/2021_also_in_2016_train"

In [ ]:
# Copy file from dir2 to dir3 if name is in dir1
for channel in channel_list:
    for filename in os.listdir(os.path.join(dir1, channel)):
        filename10_21 = '2010-2021' + filename[9:]
        file_path_copy = os.path.join(dir2, channel, filename10_21)
        filename11_21 = '2011-2021' + filename[9:]
        file_path_copy11 = os.path.join(dir2, channel, filename11_21)

        if os.path.exists(file_path_copy):
            shutil.copy(file_path_copy, os.path.join(dir3, channel))
        elif os.path.exists(file_path_copy11):
            shutil.copy(file_path_copy11, os.path.join(dir3, channel))
        #else:
            #print(f"File '{filename[9:]}' not found in {dir2}")

***
# min-max normalize data 
### Get global min, max for min-max scaling (Currently only of clean data)

In [793]:
channel_list = ['aspect','images', 'masks', 'slope', 'x_aspect', 'y_aspect'] # , 'images', 'masks', 'slope', 'x_aspect', 'y_aspect'
aspect_min = np.nan
aspect_max = np.nan
images_min = np.nan
images_max = np.nan
masks_min = np.nan
masks_max = np.nan
slope_min = np.nan
slope_max = np.nan
x_aspect_min = np.nan
x_aspect_max = np.nan
y_aspect_min = np.nan
y_aspect_max = np.nan


root_pt_list = ['data/Sophia/data_clean/data_train/data_original', 'data/Sophia/data_clean/data_test']


for channel in channel_list:
    for root in root_pt_list:
        tile_name = [item for item in os.listdir(os.path.join(root, channel))
                               if item.endswith(".tif")]
        for i, tile in enumerate(tile_name):
            tile_path =os.path.join(root, channel, tile)
            img = np.array(Image.open(tile_path))
            min_local = np.min(img)
            max_local = np.max(img)
            if i ==0: # first image cannot overwrite np.nan
                if channel == 'aspect':
                    aspect_min = min_local
                    aspect_max = max_local
                elif channel == 'images':
                    images_min = min_local
                    images_max = max_local
                elif channel == 'masks':
                    masks_min = min_local
                    masks_max = max_local
                elif channel == 'slope':
                    slope_min = min_local
                    slope_max = max_local
                elif channel == 'x_aspect':
                    x_aspect_min = min_local
                    x_aspect_max = max_local
                elif channel == 'y_aspect':
                    y_aspect_min = min_local
                    y_aspect_max = max_local
                

            else: # one has to check if local max/ min is global max/min
                if channel == 'aspect':
                    if min_local < aspect_min:
                        aspect_min = min_local
                    if max_local > aspect_max:
                        aspect_max = max_local
                elif channel == 'images':
                    if min_local < images_min:
                        images_min = min_local
                    if max_local > images_max:
                        images_max = max_local
                elif channel == 'masks':
                    if min_local < masks_min:
                        masks_min = min_local
                    if max_local > masks_max:
                        masks_max = max_local
                elif channel == 'slope':
                    if min_local < slope_min:
                        slope_min = min_local
                    if max_local > slope_max:
                        slope_max = max_local
                elif channel == 'x_aspect':
                    if min_local < x_aspect_min:
                        x_aspect_min = min_local
                    if max_local > x_aspect_max:
                        x_aspect_max = max_local
                elif channel == 'y_aspect':
                    if min_local < y_aspect_min:
                        y_aspect_min = min_local
                    if max_local > y_aspect_max:
                        y_aspect_max = max_local

In [805]:
data  = {'aspect_min': [aspect_min], 'aspect_max': [aspect_max], 'images_min': [images_min], 'images_max': [images_max], 'masks_min':[masks_min], 'masks_max':[masks_max],
         'slope_min': [slope_min], 'slope_max': [slope_max], 'x_aspect_min':[x_aspect_min],
         'x_aspect_max': [x_aspect_max],'y_aspect_min': [y_aspect_min], 'y_aspect_max': [y_aspect_max], 'x_aspect_max': [x_aspect_max], 'y_aspect_min': [y_aspect_min],'y_aspect_max': [y_aspect_max]}

In [808]:
min_max = pd.DataFrame(data)

In [809]:
min_max.to_csv('min_max.csv', index=False)

### Min max normalize

In [842]:
root_pt_list = ['data/Sophia/data_clean/data_train/data_original', 'data/Sophia/data_clean/data_train/data_original_plus_transformed',
           'data/Sophia/data_clean/data_test']
destination_pth = ['data/Sophia/data_clean/data_train/data_original_scaled', 'data/Sophia/data_clean/data_train/data_original_plus_transformed_scaled',
                  'data/Sophia/data_clean/data_test_scaled']
channel_list = ['images', 'masks', 'slope', 'x_aspect', 'y_aspect']

for root_i, root in enumerate(root_pt_list):
    for channel in channel_list:
        tile_name = [item for item in os.listdir(os.path.join(root, channel))
                               if item.endswith(".tif")]
        for i, tile in enumerate(tile_name):
            tile_path =os.path.join(root, channel, tile)
            img = np.array(Image.open(tile_path))
            if channel == 'images':
                images_min = min_max['images_min'][0]
                images_max = min_max['images_max'][0]
                img_transf = (img-images_min)/(images_max-images_min)

                path_save = os.path.join(destination_pth[root_i], channel, tile)
                Image.fromarray(img_transf).save(path_save, format="TIFF")

            elif channel == 'masks': # no mormalization needed, is label channel
                path_save = os.path.join(destination_pth[root_i], channel, tile)
                Image.fromarray(img).save(path_save, format="TIFF")

            elif channel == 'slope':
                slope_min = min_max['slope_min'][0]
                slope_max = min_max['slope_max'][0]
                img_transf = (img-slope_min)/(slope_max-slope_min)
                path_save = os.path.join(destination_pth[root_i], channel, tile)
                Image.fromarray(img_transf).save(path_save, format="TIFF")

            elif channel == 'x_aspect':
                x_aspect_min = min_max['x_aspect_min'][0]
                x_aspect_max = min_max['x_aspect_max'][0]
                img_transf = (img-x_aspect_min)/(x_aspect_max-x_aspect_min)
                path_save = os.path.join(destination_pth[root_i], channel, tile)
                Image.fromarray(img_transf).save(path_save, format="TIFF")
            elif channel == 'y_aspect':
                y_aspect_min = min_max['y_aspect_min'][0]
                y_aspect_max = min_max['y_aspect_max'][0]
                img_transf = (img-y_aspect_min)/(y_aspect_max-y_aspect_min)
                path_save = os.path.join(destination_pth[root_i], channel, tile)
                Image.fromarray(img_transf).save(path_save, format="TIFF")

### Get global mean, sd of normalized data

In [115]:
channel_list = ['images', 'slope'] #, 'x_aspect', 'y_aspect']

img_data = []
slope_data = []
x_data = []
y_data = []

path_data = ['data/Sophia/data_clean/data_test_tuktoyaktuk'] #['data/Sophia/data_clean/data_train/data_original']

for path_d in path_data:
    for channel in channel_list:
        tile_name = [item for item in os.listdir(os.path.join(path_d, channel))
                               if item.endswith(".tif")]
        for i, tile in enumerate(tile_name):
            matrix = np.array(Image.open(os.path.join(path_d, channel, tile)))            
            if channel == 'images':
                img_data.append(matrix)
            elif channel == 'slope':
                slope_data.append(matrix)
            elif channel == 'x_aspect':
                x_data.append(matrix)
            elif channel == 'y_aspect':
                y_data.append(matrix)
                                     

In [117]:
img_tot = np.concatenate(img_data)
img_sd = np.nanstd(img_tot)
img_mean = np.nanmean(img_tot)

slope_tot = np.concatenate(slope_data)
slope_sd = np.nanstd(slope_tot)
slope_mean = np.nanmean(slope_tot)

x_tot = np.concatenate(x_data)
x_sd = np.nanstd(x_tot)
x_mean = np.nanmean(x_tot)

y_tot = np.concatenate(y_data)
y_sd = np.nanstd(y_tot)
y_mean = np.nanmean(y_tot)

In [118]:
import pandas as pd

# Example data
data = {
    'img_sd': [img_sd],
    'img_mean': [img_mean],
    'slope_sd': [slope_sd],
    'slope_mean': [slope_mean],
    'x_sd': [x_sd],
    'x_mean': [x_mean],
    'y_sd': [y_sd],
    'y_mean': [y_mean]
}

# Create a DataFrame
mean_sd = pd.DataFrame(data)
mean_sd.to_csv('mean_sd_tukto.csv', index=False)

## Create directory only filled with 2021 data if it is also in 2016

In [255]:
import shutil
channel_list = ['images', 'masks', 'slope']
dir1 = "data/Sophia/data_clean/data_test"
dir2 = "data/Sophia/data_clean/2010_2021"
dir3 = "data/Sophia/data_clean/2021_also_in_2016_train"

In [257]:
for channel in channel_list:
    for filename in os.listdir(os.path.join(dir1, channel)):
        filename10_21 = '2010-2021' + filename[9:]
        file_path_copy = os.path.join(dir2, channel, filename10_21)
        filename11_21 = '2011-2021' + filename[9:]
        file_path_copy11 = os.path.join(dir2, channel, filename11_21)

        if os.path.exists(file_path_copy):
            shutil.copy(file_path_copy, os.path.join(dir3, channel))
        elif os.path.exists(file_path_copy11):
            shutil.copy(file_path_copy11, os.path.join(dir3, channel))
        #else:
            #print(f"File '{filename[9:]}' not found in {dir2}")